# Import Libraries

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import yfinance as yf
import scipy
from scipy.linalg import sqrtm, inv

import torch
import torch.nn as nn
from torch.utils.data import BatchSampler, SequentialSampler, RandomSampler
import seaborn as sns

import matplotlib
from matplotlib import pyplot as plt
import math
import os
import time
import logging

torch.set_default_tensor_type(torch.DoubleTensor)

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
print(yf.__version__)
print(np.__version__)
print(pd.__version__)
print(scipy.__version__)

print(torch.__version__)
print(sns.__version__)
print(matplotlib.__version__)
print(sklearn.__version__)

# Data Pre-processing functions

In [3]:
def fetch_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Close'].dropna()

def fetch_data_volume(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Volume'].dropna()

def calculate_changes(price_data):
    return price_data.diff()#.dropna()

def calculate_daily_returns(price_data):
    return price_data.pct_change()#.dropna()

def calculate_rate_of_change(daily_returns):
    return daily_returns.diff()#.dropna()

def create_complex_numbers(real_part, imaginary_part):
    return real_part + 1j * imaginary_part

def load_data(name, start_date = '1992-01-01', end_date = '2023-01-01'):
    print("loading data ...")
    data = fetch_data(name, start_date, end_date)
    daily_returns = calculate_daily_returns(data)
    roc = calculate_rate_of_change(daily_returns)

    diff = data.pct_change()

    data_norm = (data-data.min())/(data.max()-data.min())
    diff_norm = (diff-diff.min())/(diff.max()-diff.min())

    aligned_data = pd.concat([data_norm[1:], diff_norm], axis=1).dropna()
    aligned_data.columns = ['Close', 'Change']

    complex_data = (create_complex_numbers(aligned_data['Close'], aligned_data['Change'])).to_numpy()

    return complex_data

def split_data(data, train_ratio=0.7, val_ratio=0.15):
    data_length = len(data)
    train_end = int(data_length * train_ratio)
    val_end = int(data_length * (train_ratio + val_ratio))

    train_data = data[:train_end]
    val_data = data[train_end:val_end]
    test_data = data[val_end:]

    return torch.tensor(train_data).unsqueeze(1), torch.tensor(val_data).unsqueeze(1), torch.tensor(test_data).unsqueeze(1)


In [4]:
#@title Old load_data() functions

# used to develop best performing one
def load_data1(name, start_date = '1992-01-01', end_date = '2023-01-01'):
    print("loading data ...")
    # Fetch financial data
    data = fetch_data(name, start_date, end_date)

    # Calculate daily returns
    daily_returns = calculate_daily_returns(data)
    # Calculate rate of change of daily returns
    roc = calculate_rate_of_change(daily_returns)

    # Align data and drop missing values
    aligned_data = pd.concat([daily_returns[2:], roc[2:]], axis=1).dropna()
    aligned_data.columns = ['Day Ret', 'ROC']

    standard_scaler = StandardScaler()
    scaled = standard_scaler.fit_transform(aligned_data.values)

    scaled_features_df = pd.DataFrame(scaled, index=aligned_data.index, columns=aligned_data.columns)

    # Create complex numbers
    complex_data = (create_complex_numbers(scaled_features_df['Day Ret'], scaled_features_df['ROC'])).to_numpy()

    return complex_data

def load_data2(name, start_date = '1992-01-01', end_date = '2023-01-01'):
    print("loading data ...")
    data = fetch_data(name, start_date, end_date)
    daily_returns = calculate_daily_returns(data)
    roc = calculate_rate_of_change(daily_returns)

    diff = data.diff()

    data_norm = (data-data.mean())/ data.std() #(data-data.min())/(data.max()-data.min())
    diff_norm = (diff-diff.mean())/ diff.std() #(diff-diff.min())/(diff.max()-diff.min())

    aligned_data = pd.concat([data_norm[1:], diff_norm], axis=1).dropna()
    aligned_data.columns = ['Close', 'Change']

    complex_data = (create_complex_numbers(aligned_data['Close'], aligned_data['Change'])).to_numpy()

    return complex_data

def load_data3(name, start_date = '1992-01-01', end_date = '2023-01-01'):
    print("loading data ...")
    # Fetch financial data
    data = fetch_data(name, start_date, end_date)
    volume = fetch_data_volume(name, start_date, end_date)
    zeros = pd.Series(0, index = data.index)

    log_returns = np.log(data).diff()
    log_volumes = np.log(volume).diff()

    daily_returns = calculate_daily_returns(data)
    roc = calculate_rate_of_change(daily_returns)
    aligned_data = pd.concat([log_returns, zeros], axis=1).dropna()
    aligned_data.columns = ['Data1', 'Data2']

    standard_scaler = StandardScaler()
    scaled = standard_scaler.fit_transform(aligned_data.values)

    scaled_features_df = pd.DataFrame(scaled, index=aligned_data.index, columns=aligned_data.columns)

    # Create complex numbers
    complex_data = (create_complex_numbers(scaled_features_df['Data1'], scaled_features_df['Data2'])).to_numpy()

    #complex_data = (create_complex_numbers(aligned_data['Data1'], aligned_data['Data2'])).to_numpy()
    return complex_data

def load_data4(name, start_date = '1992-01-01', end_date = '2023-01-01'):
    print("loading data ...")
    data = fetch_data(name, start_date, end_date)
    dataset = (data-data.mean())/data.std()

    log_returns = np.log(data).diff()

    aligned_data = pd.concat([dataset[1:], log_returns[1:]], axis=1).dropna()
    aligned_data.columns = ['Data1', 'Data2']

    complex_data = (create_complex_numbers(aligned_data['Data1'], aligned_data['Data2'])).to_numpy()

    return complex_data

# Widely Linear Layer and Activation Functions

In [5]:
# define backpropagation using CR-calculus
class WidelyLinearFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, weight1, weight2, bias):
        ctx.save_for_backward(input, weight1, weight2, bias)
        z = torch.mm(input, weight1.t()) + torch.mm(input.conj(), weight2.t()) + bias
        return z

    @staticmethod
    def backward(ctx, grad_output):
        input, weight1, weight2, bias = ctx.saved_tensors
        grad_input = grad_weight1 = grad_weight2 = grad_bias = None

        if ctx.needs_input_grad[0]:
            grad_input = (grad_output.conj().mm(weight1) + grad_output.mm(weight2.conj())).conj()
        if ctx.needs_input_grad[1]:
            grad_weight1 = grad_output.t().mm(input.conj())
        if ctx.needs_input_grad[2]:
            grad_weight2 = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[3]:
            grad_bias = grad_output.sum(0)

        return grad_input, grad_weight1, grad_weight2, grad_bias


class WidelyLinearLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(WidelyLinearLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight1 = nn.Parameter(torch.complex(torch.Tensor(out_features, in_features),
                                                  torch.Tensor(out_features, in_features)))
        self.weight2 = nn.Parameter(torch.complex(torch.Tensor(out_features, in_features),
                                                  torch.Tensor(out_features, in_features)))
        if bias:
            self.bias = nn.Parameter(torch.complex(torch.Tensor(out_features),
                                                   torch.Tensor(out_features)))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight1.real, a=math.sqrt(5))#, nonlinearity='leaky_relu')
        nn.init.kaiming_uniform_(self.weight1.imag, a=math.sqrt(5))#, nonlinearity='leaky_relu')
        nn.init.kaiming_uniform_(self.weight2.real, a=math.sqrt(5))#, nonlinearity='leaky_relu')
        nn.init.kaiming_uniform_(self.weight2.imag, a=math.sqrt(5))#, nonlinearity='leaky_relu')
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight1.real)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias.real, -bound, bound)
            nn.init.uniform_(self.bias.imag, -bound, bound)

    def forward(self, input):
        # z = torch.mm(input, self.weight1.t()) + torch.mm(input.conj(), self.weight2.t()) + self.bias
        # return z
        return WidelyLinearFunction.apply(input, self.weight1, self.weight2, self.bias)

class ComplexArctan(torch.autograd.Function):
    @staticmethod
    def forward(ctx, z):
        ctx.save_for_backward(z)
        output = torch.atan(z)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        z, = ctx.saved_tensors
        grad_input = (grad_output.conj() / (1 + torch.square(z)))
        return grad_input.conj()

class ComplexSigmoid(torch.autograd.Function):
    @staticmethod
    def forward(ctx, z):
        ctx.save_for_backward(z)
        output = torch.sigmoid(z)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        z, = ctx.saved_tensors
        grad_input = grad_output.conj() * torch.sigmoid(z) * (1 - torch.sigmoid(z))
        return grad_input.conj()

class ComplexActivation(nn.Module):
    def forward(self, input):
        return torch.atan(input)
        #return ComplexArctan.apply(input)
        #return ComplexSigmoid.apply(input)

class ComplexRelu(nn.Module):
    def forward(self, input):
        f = nn.ReLU()
        z = torch.view_as_real(input)
        f_z = f(z)
        return torch.view_as_complex(f_z)

# class ComplexBatchNorm1d(nn.Module):
#     def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True):
#         super(ComplexBatchNorm1d, self).__init__()
#         self.bn_r = nn.BatchNorm1d(num_features, eps=eps, momentum=momentum, affine=affine)
#         self.bn_i = nn.BatchNorm1d(num_features, eps=eps, momentum=momentum, affine=affine)

#     def forward(self, input):
#         output = self.bn_r(input.real) + 1j*self.bn_i(input.imag)
#         return output

## Gradient checks

In [6]:
from torch.autograd import gradcheck

widely = WidelyLinearFunction.apply

input = (torch.randn(100,1,dtype=torch.cdouble,requires_grad=True), torch.randn(128,1,dtype=torch.cdouble,requires_grad=True), torch.randn(128,1,dtype=torch.cdouble,requires_grad=True), torch.randn(128,dtype=torch.cdouble,requires_grad=True))
test = gradcheck(widely, input, eps=1e-6, atol=1e-4)
print(test)

True


In [7]:
from torch.autograd import gradcheck

# change activation to test
activation = ComplexSigmoid.apply #ComplexArctan.apply

input = torch.randn(100,1,dtype=torch.cdouble,requires_grad=True)
test = gradcheck(activation, input, eps=1e-6, atol=1e-4)
print(test)

True


# CDCCA Loss

In [6]:
class cdcca_loss():
    def __init__(self, outdim_size, device):
        self.outdim_size = outdim_size
        self.device = device

    def loss(self, H1, H2):

        r1 = 1e-3 + 1j*1e-5
        r2 = 1e-3 + 1j*1e-5
        eps = 1e-3

        H1, H2 = H1.t(), H2.t()

        o1 = o2 = H1.size(0)
        m = H1.size(1)

        H1bar = H1 - H1.mean(dim=1).unsqueeze(dim=1)
        H2bar = H2 - H2.mean(dim=1).unsqueeze(dim=1)

        # Compute covariance matrices and add regularization term so they are positive definite
        SigmaHat12 = (1.0 / (m - 1)) * torch.matmul(H1bar, H2bar.conj().t())
        SigmaHat11 = (1.0 / (m - 1)) * torch.matmul(H1bar, H1bar.conj().t()) + r1 * torch.eye(o1, device=self.device)
        SigmaHat22 = (1.0 / (m - 1)) * torch.matmul(H2bar, H2bar.conj().t()) + r2 * torch.eye(o2, device=self.device)

        SigmaTilde12 = (1.0 / (m - 1)) * torch.matmul(H1bar, H2bar.t())
        SigmaTilde11 = (1.0 / (m - 1)) * torch.matmul(H1bar, H1bar.t()) + r1 * torch.eye(o1, device=self.device)
        SigmaTilde22 = (1.0 / (m - 1)) * torch.matmul(H2bar, H2bar.t()) + r2 * torch.eye(o2, device=self.device)

        AugmentedSigma12 = torch.cat((torch.cat((SigmaHat12, SigmaTilde12), 1),torch.cat((torch.conj(SigmaTilde12), torch.conj(SigmaHat12)), 1)), 0)
        AugmentedSigma11 = torch.cat((torch.cat((SigmaHat11, SigmaTilde11), 1),torch.cat((torch.conj(SigmaTilde11), torch.conj(SigmaHat11)), 1)), 0)
        AugmentedSigma22 = torch.cat((torch.cat((SigmaHat22, SigmaTilde22), 1),torch.cat((torch.conj(SigmaTilde22), torch.conj(SigmaHat22)), 1)), 0)

        [D1, V1] = torch.linalg.eig(AugmentedSigma11)
        [D2, V2] = torch.linalg.eig(AugmentedSigma22)

        AugSigma11RootInv = torch.matmul(torch.matmul(V1, torch.diag(D1 ** -0.5)), V1.conj().t())
        AugSigma22RootInv = torch.matmul(torch.matmul(V2, torch.diag(D2 ** -0.5)), V2.conj().t())

        Tval = torch.matmul(torch.matmul(AugSigma11RootInv, AugmentedSigma12), AugSigma22RootInv)

        C = (1/np.sqrt(2)) * torch.cat((torch.cat((torch.eye(o1, device=self.device), torch.complex(torch.zeros(o1, o1, device = self.device), torch.eye(o1, device=self.device))), 1),
                                        torch.cat((torch.eye(o1, device=self.device), torch.complex(torch.zeros(o1, o1, device = self.device), -torch.eye(o1, device=self.device))), 1)), 0)

        T_hat = torch.matmul(torch.matmul(C.conj().t(), Tval), C)
        T_hat = T_hat.real

        [_, K, _] = torch.linalg.svd(T_hat)

        corr = 1/(o1*2) * torch.sum(K)

        return - corr.real

# MLP Network and ComplexDeepCCA

In [7]:
class MlpNetwork(nn.Module):
    def __init__(self, layer_sizes, input_size):
        super(MlpNetwork, self).__init__()
        layers = []
        layer_sizes = [input_size] + layer_sizes
        self.complex_activation = ComplexRelu()
        for l_id in range(len(layer_sizes) - 1):
            if l_id == len(layer_sizes) - 2:
                layers.append(nn.Sequential(
                    #ComplexBatchNorm1d(num_features=layer_sizes[l_id], affine=False),
                    WidelyLinearLayer(layer_sizes[l_id], layer_sizes[l_id + 1]),
                ))
            else:
                layers.append(nn.Sequential(
                    WidelyLinearLayer(layer_sizes[l_id], layer_sizes[l_id + 1]),
                    self.complex_activation,
                    #ComplexBatchNorm1d(num_features=layer_sizes[l_id + 1], affine=False),
                ))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class ComplexDeepCCA(nn.Module):
    def __init__(self, layer_sizes1, layer_sizes2, input_size1, input_size2, outdim_size, device=torch.device('cpu')):
        super(ComplexDeepCCA, self).__init__()
        self.model1 = MlpNetwork(layer_sizes1, input_size1).double()
        self.model2 = MlpNetwork(layer_sizes2, input_size2).double()

        self.loss = cdcca_loss(outdim_size, device).loss

    def forward(self, x1, x2, device):
        input1 = torch.randn(x1.size(0),1, dtype=torch.cdouble)
        input2 = torch.randn(x2.size(0),1, dtype=torch.cdouble)

        for i in range(0,len(input1)):
            input1[i] = torch.complex(x1[i][0][0], x1[i][0][1])
            input2[i] = torch.complex(x2[i][0][0], x2[i][0][1])

        output1 = self.model1(input1.to(device=device))
        output2 = self.model2(input2.to(device=device))

        return output1, output2

# Linear CCA

In [8]:
class linear_cca():
    def __init__(self):
        self.w = [None, None]
        self.m = [None, None]

    def fit(self, H1, H2, outdim_size):

        r1 = 1e-4 + 1j*1e-6
        r2 = 1e-4 + 1j*1e-6

        m = H1.shape[0]
        o1 = H1.shape[1]
        o2 = H2.shape[1]

        self.m[0] = np.mean(H1, axis=0)
        self.m[1] = np.mean(H2, axis=0)
        H1bar = H1 - np.tile(self.m[0], (m, 1))
        H2bar = H2 - np.tile(self.m[1], (m, 1))

        SigmaHat12 = (1.0 / (m - 1)) * np.dot(H1bar.conjugate().T, H2bar)
        SigmaHat11 = (1.0 / (m - 1)) * np.dot(H1bar.conjugate().T, H1bar) + r1 * np.identity(o1)
        SigmaHat22 = (1.0 / (m - 1)) * np.dot(H2bar.conjugate().T, H2bar) + r2 * np.identity(o2)

        SigmaTilde12 = (1.0 / (m - 1)) * np.dot(H1bar.T, H2bar)
        SigmaTilde11 = (1.0 / (m - 1)) * np.dot(H1bar.T, H1bar) + r1 * np.identity(o1)
        SigmaTilde22 = (1.0 / (m - 1)) * np.dot(H2bar.T, H2bar) + r2 * np.identity(o2)

        AugmentedSigma12 = np.concatenate((np.concatenate((SigmaHat12, SigmaTilde12), axis=1),
                                           np.concatenate((SigmaTilde12.conjugate(), SigmaHat12.conjugate()), axis=1)), axis = 0)
        AugmentedSigma11 = np.concatenate((np.concatenate((SigmaHat11, SigmaTilde11), axis=1),
                                           np.concatenate((SigmaTilde11.conjugate(), SigmaHat11.conjugate()), axis=1)), axis = 0)
        AugmentedSigma22 = np.concatenate((np.concatenate((SigmaHat22, SigmaTilde22), axis=1),
                                           np.concatenate((SigmaTilde22.conjugate(), SigmaHat22.conjugate()), axis=1)), axis = 0)

        [D1, V1] = scipy.linalg.eig(AugmentedSigma11)
        [D2, V2] = scipy.linalg.eig(AugmentedSigma22)

        AugSigma11RootInv = np.dot(np.dot(V1, np.diag(D1 ** -0.5)), V1.conjugate().T)
        AugSigma22RootInv = np.dot(np.dot(V2, np.diag(D2 ** -0.5)), V2.conjugate().T)

        Tval = np.dot(np.dot(AugSigma11RootInv, AugmentedSigma12), AugSigma22RootInv)

        C = (1/np.sqrt(2)) * np.concatenate((np.concatenate((np.identity(o1), 1j*np.identity(o1)), axis=1),
                                             np.concatenate((np.identity(o1), -1j*np.identity(o1)), axis=1)), axis = 0)

        T_hat = np.dot(np.dot(C.conjugate().T, Tval), C)
        T_hat = T_hat.real

        [U_hat, K, V_hat] = scipy.linalg.svd(T_hat)
        V_hat = V_hat.conjugate().T

        U = np.dot(np.dot(C, U_hat), C.conjugate().T)
        V = np.dot(np.dot(C, V_hat), C.conjugate().T)

        A = np.dot(U.conjugate().T, AugSigma11RootInv)
        B = np.dot(V.conjugate().T, AugSigma22RootInv)

        self.w[0] = [A[0:outdim_size, 0:outdim_size], A[0:outdim_size, outdim_size:A.shape[1]]]
        self.w[1] = [B[0:outdim_size, 0:outdim_size], B[0:outdim_size, outdim_size:A.shape[1]]]

    def _get_result(self, x, idx):
        input = x - self.m[idx].reshape([1, -1]).repeat(len(x), axis=0)
        input_conj = input.conjugate()
        result = np.dot(input, self.w[idx][0]) + np.dot(input_conj, self.w[idx][1])
        return result

    def transform(self, H1, H2):
        return self._get_result(H1, 0), self._get_result(H2, 1)

# Trainer

In [9]:
torch.set_default_tensor_type(torch.DoubleTensor)

class Trainer():
    def __init__(self, model, linear_cca, outdim_size, epoch_num, batch_size, learning_rate, reg_par, device=torch.device('cpu')):
        self.model = nn.DataParallel(model) #nn.parallel.DistributedDataParallel(model)
        self.model.to(device)
        self.epoch_num = epoch_num
        self.batch_size = batch_size
        self.loss = model.loss
        self.optimizer = torch.optim.RMSprop(self.model.parameters(), lr=learning_rate, weight_decay=reg_par)
        # self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.device = device

        self.linear_cca = linear_cca
        self.outdim_size = outdim_size

        #create a logger
        self.logger = logging.getLogger('mylogger')
        level = logging.INFO #logging.DEBUG
        self.logger.setLevel(level)

        if (self.logger.hasHandlers()):
            self.logger.handlers.clear()

        handler = logging.FileHandler('DCCA.log')
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)
        self.logger.propagate = False

        ch = logging.StreamHandler()
        ch.setLevel(level)
        self.logger.addHandler(ch)

        self.logger.info(self.model)
        self.logger.info("Number of model parameters: {}".format(len(list(self.model.parameters()))))
        self.logger.info("Model Parameters:")
        for i in range(len(list(self.model.parameters()))):
            self.logger.info(list(self.model.parameters())[i].size())

        self.logger.info(self.optimizer)

    def train(self, x1, x2, vx1=None, vx2=None, tx1=None, tx2=None, checkpoint='checkpoint.model'):
        """
        x1, x2 are the vectors needs to be make correlated
        dim=[batch_size, feats]
        """
        x1.to(self.device, dtype = torch.cfloat)
        x2.to(self.device, dtype = torch.cfloat)

        data_size = x1.size(0)

        if vx1 is not None and vx2 is not None:
            best_val_loss = 0
            vx1.to(self.device)
            vx2.to(self.device)
        if tx1 is not None and tx2 is not None:
            tx1.to(self.device)
            tx2.to(self.device)

        train_losses = []
        train_loss_history = []
        val_loss_history = []
        for epoch in range(self.epoch_num):
            epoch_start_time = time.time()
            self.model.train()
            batch_idxs = list(BatchSampler(SequentialSampler(range(data_size)), batch_size=self.batch_size, drop_last=False)) #RandomSampler #SequentialSampler
            #with torch.autograd.profiler.profile() as prof:
            for batch_idx in batch_idxs:
                self.optimizer.zero_grad()
                batch_x1 = x1[batch_idx, :]
                batch_x2 = x2[batch_idx, :]
                self.optimizer.zero_grad()
                out1, out2 = self.model(batch_x1, batch_x2, self.device)
                loss = self.loss(out1, out2)
                train_losses.append(loss.item())
                loss.backward()
                self.optimizer.step()
            train_loss = np.mean(train_losses)
            #print(prof.key_averages().table(sort_by="self_cpu_time_total"))

            info_string = "Epoch {:d}/{:d} - time: {:.2f} - training_loss: {:.4f}"
            if vx1 is not None and vx2 is not None:
                with torch.no_grad():
                    self.model.eval()
                    val_loss = self.transform(vx1, vx2)
                    info_string += " - val_loss: {:.4f}".format(-val_loss)
                    if val_loss < best_val_loss:
                        self.logger.info(
                            "Epoch {:d}: val_loss improved from {:.4f} to {:.4f}, saving model to {}".format(epoch + 1, -best_val_loss, -val_loss, checkpoint))
                        best_val_loss = val_loss
                        torch.save(self.model.state_dict(), checkpoint)
                    else:
                        self.logger.info("Epoch {:d}: val_loss did not improve from {:.4f}".format(epoch + 1, -best_val_loss))
            else:
                torch.save(self.model.state_dict(), checkpoint)
            epoch_time = time.time() - epoch_start_time
            self.logger.info(info_string.format(epoch + 1, self.epoch_num, epoch_time, -train_loss))

            train_loss_history.append(train_loss)
            val_loss_history.append(val_loss)

        # train_linear_cca
        if self.linear_cca is not None:
            _, outputs = self._get_outputs(x1, x2)
            self.linear_cca.fit(outputs[0], outputs[1], self.outdim_size)

        checkpoint_ = torch.load(checkpoint)
        self.model.load_state_dict(checkpoint_)

        if vx1 is not None and vx2 is not None:
            loss = self.transform(vx1, vx2)
            self.logger.info("loss on validation data: {:.4f}".format(-loss))

        if tx1 is not None and tx2 is not None:
            loss = self.transform(tx1, tx2)
            self.logger.info('loss on test data: {:.4f}'.format(-loss))

        fig = plt.figure(figsize=(10,4))
        epochs = np.arange(1, self.epoch_num+1, step=1)
        plt.plot(epochs, -1 * np.array(train_loss_history), color = 'tab:blue', label='Training')
        plt.plot(epochs, -1 * np.array(val_loss_history), color = 'tab:orange', label='Validation')
        plt.xticks(epochs)
        plt.title('Model Correlation')
        plt.ylabel('Correlation')
        plt.xlabel('Epochs')
        plt.legend()
        plt.grid()

        return [-1 * np.array(train_loss_history), -1 * np.array(val_loss_history)]

    def transform(self, x1, x2, use_linear_cca=False):
        with torch.no_grad():
            losses, outputs = self._get_outputs(x1, x2)

            if use_linear_cca:
                print("Linear CCA started!")
                outputs = self.linear_cca.transform(outputs[0], outputs[1])
                return losses, outputs
                # return np.mean(losses), outputs
            else:
                return np.mean(losses)

    def _get_outputs(self, x1, x2):
        with torch.no_grad():
            self.model.eval()
            data_size = x1.size(0)
            batch_idxs = list(BatchSampler(SequentialSampler(range(data_size)), batch_size=self.batch_size, drop_last=False))
            losses = []
            outputs1 = []
            outputs2 = []
            for batch_idx in batch_idxs:
                batch_x1 = x1[batch_idx, :]
                batch_x2 = x2[batch_idx, :]
                out1, out2 = self.model(batch_x1, batch_x2, self.device)
                outputs1.append(out1)
                outputs2.append(out2)
                loss = self.loss(out1, out2)
                losses.append(loss.item())
        outputs = [torch.cat(outputs1, dim=0).cpu().numpy(),
                   torch.cat(outputs2, dim=0).cpu().numpy()]
        return losses, outputs

# MAIN

## Parameters Section

In [10]:
############

device = torch.device('cuda')
# print("Using", torch.cuda.device_count(), "GPUs")
# print("GPU Name:", torch.cuda.get_device_name())

# the path to save the final learned features
save_to = './new_features.gz'

# the size of the new space learned by the model (number of the new features)
outdim_size = 3

# size of the input for view 1 and view 2
input_shape1 = input_shape2 = 1

# number of layers with nodes in each one
# 5 components
# layer_sizes1 = [256, 512, 1024, 1024, 1024, 512, outdim_size]
# layer_sizes2 = [256, 512, 1024, 1024, 1024, 512, outdim_size]

# 3 components
layer_sizes1 = [128, 256, 512, 1024, outdim_size]
layer_sizes2 = [128, 256, 512, 1024, outdim_size]

# 2 components
# layer_sizes1 = [128, 256, 512, outdim_size]
# layer_sizes2 = [128, 256, 512, outdim_size]

# the parameters for training the network
learning_rate = 1e-3
epoch_num = 10
batch_size = 50

# the regularization parameter of the network
# necessary to avoid the gradient exploding/vanishing
reg_par = 1e-5

# if a linear CCA should get applied on the learned features
apply_linear_cca = True

############

## Data Pre-processing

In [ ]:
# Load data
# tick1 = 'AAPL'
# tick2 = 'PFE'
tick1 = '^GSPC'
tick2 = '^DJI'
start = '1992-01-01'
end = '2023-01-01'

data1 = fetch_data(tick1, start, end)
data2 = fetch_data(tick2, start, end)

print(data1.shape)
print(data2.shape)

data1_norm = (data1-data1.mean())/data1.std()
data2_norm = (data2-data2.mean())/data2.std()

train_ratio=0.7
val_ratio=0.15
data_length = len(data1)
train_end = int(data_length * train_ratio)
val_end = int(data_length * (train_ratio + val_ratio))

idx1 = data1.index
idx2 = data2.index

# Plot complex data
fig, ax = plt.subplots(3, 2)
fig.set_size_inches(10,9)
fig.tight_layout()
fig.subplots_adjust(hspace = 0.4, wspace = 0.2)

ax[0][0].plot(data1)
ax[0][0].axvline(x = idx1[train_end], alpha = 0.8, color='r', linewidth = 2, linestyle = '--')
ax[0][0].axvline(x = idx1[val_end], alpha = 0.8, color='g', linewidth = 2, linestyle = '--')
ax[0][0].set_xlabel('Date')
ax[0][0].set_ylabel('Closing Price')
ax[0][0].set_title('Data1')
ax[0][0].grid(True)

ax[0][1].plot(data2)
ax[0][1].axvline(x = idx2[train_end], alpha = 0.8, color='r', linewidth = 2, linestyle = '--')
ax[0][1].axvline(x = idx2[val_end], alpha = 0.8, color='g', linewidth = 2, linestyle = '--')
ax[0][1].set_xlabel('Date')
ax[0][1].set_ylabel('Closing Price')
ax[0][1].set_title('Data2')
ax[0][1].grid(True)

ax[1][0].plot(data1.pct_change())
ax[1][0].axvline(x = idx2[train_end], alpha = 0.8, color='r', linewidth = 2, linestyle = '--')
ax[1][0].axvline(x = idx2[val_end], alpha = 0.8, color='g', linewidth = 2, linestyle = '--')
ax[1][0].set_xlabel('Date')
ax[1][0].set_ylabel('ROC')
ax[1][0].set_title('Data1')
ax[1][0].grid(True)

ax[1][1].plot(data2.pct_change())
ax[1][1].axvline(x = idx2[train_end], alpha = 0.8, color='r', linewidth = 2, linestyle = '--')
ax[1][1].axvline(x = idx2[val_end], alpha = 0.8, color='g', linewidth = 2, linestyle = '--')
ax[1][1].set_xlabel('Date')
ax[1][1].set_ylabel('ROC')
ax[1][1].set_title('Data2')
ax[1][1].grid(True)

view1 = load_data(tick1, start, end)
view2 = load_data(tick2, start, end)

circ_view1 = np.abs(np.mean(view1**2)/(np.mean(np.abs(view1**2)))); # Circularity coefficient of data
ax[2][0].scatter(view1.real, view1.imag, marker='o', alpha=0.5)
ax[2][0].set_xlabel('Real Part')
ax[2][0].set_ylabel('Imaginary Part')
ax[2][0].set_title('View 1 - Complex Data, \u03B7 ={}'.format(round(circ_view1, 4)))
ax[2][0].grid(True)

circ_view2 = np.abs(np.mean(view2**2)/(np.mean(np.abs(view2**2)))); # Circularity coefficient of data
ax[2][1].scatter(view2.real, view2.imag, marker='o', alpha=0.5)
ax[2][1].set_xlabel('Real Part')
ax[2][1].set_ylabel('Imaginary Part')
ax[2][1].set_title('View 2 - Complex Data, \u03B7 ={}'.format(round(circ_view2, 4)))
ax[2][1].grid(True)

# Split data
train1, val1, test1 = split_data(view1, train_ratio=train_ratio, val_ratio=val_ratio)
train2, val2, test2 = split_data(view2, train_ratio=train_ratio, val_ratio=val_ratio)

fig.savefig(('indices.pdf'), bbox_inches='tight')

## Model Training

In [ ]:
# Building, training, and producing the new features by CDCCA
model = ComplexDeepCCA(layer_sizes1, layer_sizes2, input_shape1, input_shape2, outdim_size, device=device).double()
l_cca = None
if apply_linear_cca:
    l_cca = linear_cca()
trainer = Trainer(model, l_cca, outdim_size, epoch_num, batch_size, learning_rate, reg_par, device=device)

history = trainer.train(train1, train2, val1, val2, test1, test2)

In [ ]:
# run this cell to visualize the
fig = plt.figure(figsize=(10,4))
epochs = np.arange(1, epoch_num+1, step=1)
epochs1 = np.arange(0, epoch_num+9, step=10)
plt.plot(epochs, history[0], color = 'tab:blue', label='Training')
plt.plot(epochs, history[1], color = 'tab:orange', label='Validation')
plt.xticks(epochs1)
plt.xlim(0, 52)
plt.title('Model Correlation')
plt.ylabel('Correlation')
plt.xlabel('Epochs')
plt.legend()
plt.grid()

## Canonical Variables and Correlation (loss)

In [ ]:
# transform the views
train_loss1, train_outputs = trainer.transform(train1, train2, apply_linear_cca)
val_loss1, val_outputs = trainer.transform(val1, val2, apply_linear_cca)
test_loss1, test_outputs = trainer.transform(test1, test2, apply_linear_cca)

print("Training data canonical correlation", -np.mean(train_loss1))
print("Validation data canonical correlation", -np.mean(val_loss1))
print("Testing data canonical correlation", -np.mean(test_loss1))

In [ ]:
output1 = [train_outputs[0], val_outputs[0], test_outputs[0]]
output2 = [train_outputs[1], val_outputs[1], test_outputs[1]]

dataset = ["Training", "Validation", "Testing"]

# create figures for Real parts vs each each other
fig = plt.figure(constrained_layout=True, figsize=(25, 10))
subfigs = fig.subfigures(nrows=3, ncols=1)

for row, subfig in enumerate(subfigs):
    subfig.suptitle(f'{dataset[row]}', fontweight="bold")

    # create 1x3 subplots per subfig
    axs = subfig.subplots(nrows=1, ncols=outdim_size)
    for col, ax in enumerate(axs):
        ax.scatter(output1[row][:, col].real, output2[row][:, col].real)

        corr = np.corrcoef(output1[row][:, col].real, output2[row][:, col].real)[0, 1]
        ax.set_title("Component {}".format(col + 1))
        # ax.set_title("Component {}, corr = {:.2f}".format(col + 1, corr))
        #ax.set_title("Component {}, corr = |{:.2f}|e^({:.2f}j)".format(col + 1, mag, phase))
        #ax.set_title("Component {}".format(col + 1))
        ax.set_xlabel('View 1')
        ax.set_ylabel('View 2')
        ax.grid()

In [ ]:
# create figures for Imaginary parts vs each each other
fig = plt.figure(constrained_layout=True, figsize=(15, 10))
subfigs = fig.subfigures(nrows=3, ncols=1)

for row, subfig in enumerate(subfigs):
    subfig.suptitle(f'{dataset[row]}', fontweight="bold")

    # create 1x3 subplots per subfig
    axs = subfig.subplots(nrows=1, ncols=outdim_size)
    for col, ax in enumerate(axs):
        ax.scatter(output1[row][:, col].imag, output2[row][:, col].imag)

        ax.set_title("Component {}".format(col + 1))
        ax.set_xlabel('View 1')
        ax.set_ylabel('View 2')
        ax.grid()

In [ ]:
#@title Plotting trials
# create figures for Real vs Imaginary parts vs each each other
fig = plt.figure(constrained_layout=True, figsize=(15, 10))
subfigs = fig.subfigures(nrows=3, ncols=1)

for row, subfig in enumerate(subfigs):
    subfig.suptitle(f'{dataset[row]}', fontweight="bold")

    # create 1x3 subplots per subfig
    axs = subfig.subplots(nrows=1, ncols=outdim_size)
    for col, ax in enumerate(axs):
        # ax.scatter(output1[row][:, col].real, output2[row][:, col].real)
        ax.scatter(output1[row][:, col].real, output1[row][:, col].imag, marker='o', alpha=0.5, label = 'View 1')
        ax.scatter(output2[row][:, col].real, output2[row][:, col].imag, marker='o', alpha=0.5, label = 'View 2')
        ax.set_title("Component {}".format(col + 1))
        ax.set_xlabel('Real')
        ax.set_ylabel('Imaginary')
        ax.legend()
        ax.grid()

In [ ]:
d = torch.load('checkpoint.model')
trainer.model.load_state_dict(d)
trainer.model.parameters()

<generator object Module.parameters at 0x7f1844145d90>